In [1]:
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
influence_pd=pd.read_csv('influence_data.csv')
artist_pd=pd.read_csv('data_by_artist.csv')
artist_pd=artist_pd.drop(['mode','key','duration_ms','popularity','count'],axis=1)
artist_pd

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,acousticness,instrumentalness,liveness,speechiness
0,Frank Sinatra,792507,0.384478,0.238017,0.364288,110.181698,-14.271141,0.735648,0.020855,0.232106,0.049614
1,Vladimir Horowitz,119107,0.343210,0.118844,0.225951,94.900679,-23.193418,0.990070,0.879508,0.183812,0.043360
2,Johnny Cash,816890,0.619803,0.449381,0.680662,115.037747,-11.593104,0.685637,0.022647,0.242243,0.098216
3,Billie Holiday,79016,0.572637,0.201368,0.498934,109.912172,-13.225966,0.908499,0.013064,0.217727,0.062432
4,Bob Dylan,66915,0.512598,0.477932,0.551934,126.160149,-11.184330,0.562567,0.034211,0.308978,0.064535
...,...,...,...,...,...,...,...,...,...,...,...
5849,Natalie La Rose,3359519,0.830000,0.520000,0.735000,104.990000,-8.714000,0.000792,0.000013,0.065600,0.037600
5850,Sarah Ross,3381566,0.721000,0.944000,0.626000,85.002000,-5.982000,0.013000,0.000000,0.320000,0.159000
5851,Rotimi,3410250,0.637000,0.501000,0.431000,103.993000,-6.148000,0.229000,0.000059,0.099000,0.187000
5852,Jillian Jacqueline,3455945,0.547000,0.672000,0.283000,155.791000,-5.023000,0.304000,0.000000,0.099600,0.049600


In [3]:
for col in artist_pd.columns:
    if(col=='artist_name' or col=='artist_id'): continue
    artist_pd[col]=(artist_pd[col]-artist_pd[col].mean())/(artist_pd[col].std())

In [4]:
artist_attr={}
for i in range(artist_pd.shape[0]):
    artist_attr[artist_pd.loc[i].artist_id]=artist_pd.loc[i].drop('artist_id').to_dict()

In [5]:
edge=[]
id_to_num={}
num_to_id={}
ou_deg=[{'id':i,'val':0,'cnti':0,'cnto':0,'vali':0,} for i in range(6000)]
head=[-1 for i in range(6000)]
def add(u,v,w):
    edge.append({'from':u,'to':v,'w':w,'next':head[u]})
    head[u]=len(edge)-1
    return
def cal(u,v):
    atu=artist_attr[u]
    atv=artist_attr[v]
    n,sum=0,0
    for key in atu.keys():
        if(key=='artist_name'): continue
        n+=1
        sum+=math.pow((atu[key]-atv[key]),2)
    return 1/math.sqrt(sum/n)

In [6]:
for i in range(influence_pd.shape[0]):
    u=influence_pd.loc[i].influencer_id
    v=influence_pd.loc[i].follower_id
    if(artist_attr.setdefault(v)==None): continue
    if(id_to_num.setdefault(u)==None): 
        id_to_num[u]=len(id_to_num)+1
        num_to_id[id_to_num[u]]=u
    if(id_to_num.setdefault(v)==None): 
        id_to_num[v]=len(id_to_num)+1
        num_to_id[id_to_num[v]]=v
    w=cal(u,v)
    u=id_to_num[u]
    v=id_to_num[v]
    add(u,v,w)
    ou_deg[u]['cnto']+=1
    ou_deg[v]['cnti']+=1
    ou_deg[v]['vali']+=w
    ou_deg[u]['val']+=w

In [19]:
ou_deg=sorted(ou_deg,key=lambda x:x['val'],reverse=True)
rank=[0 for i in range(6000)]
for i in range(len(ou_deg)):
    if(ou_deg[i]['id']==0):
        print(i)
        break
    rank[ou_deg[i]['id']]=i+1

3774


In [10]:
for i in range(100):
    print(artist_attr[num_to_id[ou_deg[i]['id']]]['artist_name']+","+str(ou_deg[i]['val'])+","+str(ou_deg[i]['cnto'])+","+str(ou_deg[i]['vali'])+","+str(ou_deg[i]['cnti']))

The Beatles,1071.1754143201426,614,69.04370221472512,31
Bob Dylan,555.9508900073295,389,38.31470345054155,29
The Rolling Stones,510.71873520773806,319,53.43463108031689,39
Led Zeppelin,385.7045734465682,221,38.89669539105694,24
David Bowie,363.89083478020495,238,48.19144023289242,25
The Kinks,333.7314872509998,191,15.419298497662767,8
The Beach Boys,318.9717216948843,185,25.59837905619626,13
Jimi Hendrix,309.29802302480135,201,40.81794065154469,32
The Who,288.7611227165087,149,28.69540841416305,20
The Byrds,275.7731158127913,158,41.52308257673377,20
Marvin Gaye,274.25679009397294,169,27.9104145175812,18
Hank Williams,257.803109913891,184,9.502515134603058,3
James Brown,241.71783665067693,154,21.973805997434315,15
The Velvet Underground,240.99327691103906,181,12.557787453917276,8
Elvis Presley,237.2617198445996,166,27.967582970122482,16
Stevie Wonder,231.44169683714935,138,17.149757151272894,11
Black Sabbath,227.509327336247,171,21.96143373684375,12
Michael Jackson,216.96814277318205,10

In [18]:
ch={}
def show(x,w,b):
    print("-"*b+artist_attr[num_to_id[x]]['artist_name']+" "+str(rank[x]))
    if(ch.setdefault(x)==None): return
    for i in range(len(ch[x])):
        show(ch[x][i]['id'],ch[x][i]['val'],b+2)
index=1000
u=ou_deg[index]['id'] #
lim=[4,2]
q=[]
q.append([u,0])
# print(" "*2*q[0][1]+artist_attr[num_to_id[q[0][0]]]['artist_name'])
while(len(q)!=0):
    u=q[0][0]
    b=q[0][1]+1
    q.pop(0)
    i=head[u]
    children_list=[]
    cnt=0
    while(i!=-1):
        v=edge[i]['to']
        if(num_to_id.setdefault(v)==None or artist_attr.setdefault(num_to_id[v])==None):
            i=edge[i]['next']
            continue
        children_list.append({'id':edge[i]['to'],'val':cal(num_to_id[u],num_to_id[v])})
        i=edge[i]['next']
    
    children_list=sorted(children_list,key=lambda x:x['val'],reverse=True)
    ch[u]=[]
    for i in range(min(lim[b-1],len(children_list))): 
        ch[u].append({'id':children_list[i]['id'],'val':children_list[i]['val']})
        if(b!=2):
            q.append([children_list[i]['id'],b])
            
show(ou_deg[index]['id'],0,0)

Aztec Camera 1001
--Del Amitri 1590
----KT Tunstall 5106
----Ray LaMontagne 2614
--Phoenix 2116
----Foster the People 5402
----Cherub 5461
--The Lightning Seeds 1578
----The Cardigans 1108
----Gorillaz 3211
--Texas 1695
----The Cardigans 1108
----KT Tunstall 5106


In [ ]:
music_pd=pd.read_csv('full_music_data.csv')
music_pd=music_pd.drop(['artist_names','mode','key','explicit','popularity','year','release_date','song_title (censored)'],axis=1)
music_pd

In [ ]:
for col in music_pd.columns:
    if(col=='artist_names' or col=='artists_id'): continue
    music_pd[col]=(music_pd[col]-music_pd[col].mean())/(music_pd[col].std())
music_pd

In [ ]:
artist_music_attr={}
for i in range(music_pd.shape[0]):
    cur=music_pd.loc[i]
    for artist_id in cur['artists_id']:
        if(artist_music_attr.setdefault(artist_id)==None):
            